## Исследование надёжности заёмщиков

Заказчик: кредитный отдел банка.

Цель: выяснить как влияет на возвратность кредитов семейное положение заёмщиков, наличие детей и их количество, уровень дохода и цель кредита.

Входные данные от банка: статистика о платёжеспособности клиентов.

## Оглавление 

1. [Обзор и первичное исследование данных](#description)

    1.1 [Описание данных](#description)

    1.2 [Открытие данных](#start)

    1.3 [Изучение общей информации](#main_info)
    
    1.4 [Проверка корректность наименований столбцов](#columns_info)
    
    1.5 [Изучение внешнего вида таблицы](#main_view)
    
    1.6 [Изучение статистических данных](#stat_info)
    
    1.7 [Дубликаты и уникальные значения в столбцах](#unique_info)
    
    1.8 [Вывод](#main_conclusion)
    
    
2. [Предобработка данных](#preprocessing)

    2.1 [Артефакты](#artifact)
    
    2.2 [Обработка пропущенных значений](#null)
    
    2.3 [Замена типа данных](#type)
    
    2.4 [Обработка дубликатов](#duplicates)
    
    2.5 [Лемматизация](#lemma)
    
    2.6 [Категоризация](#category)
    
    
3. [Ответы на вопросы](#answers)

    3.1 [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#answers_child)

    3.2 [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#answers_family)

    3.3 [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#answers_income)

    3.4 [Как разные цели кредита влияют на его возврат в срок?](#answers_purpose)


4.[Общий вывод](#conclusion)

##  1. Обзор и первичное исследование данных

**1.1 Описание данных** <a id="description"></a>

children — количество детей в семье 

days_employed — трудовой стаж в днях 

dob_years — возраст клиента в годах 

education — образование клиента 

education_id — идентификатор образования 

family_status — семейное положение 

family_status_id — идентификатор семейного положения 

gender — пол клиента 

income_type — тип занятости 

debt — имел ли задолженность по возврату кредитов 

total_income — доход в месяц 

purpose — цель получения кредита

**1.2 Открытие данных** <a id="start"></a>

Импортируем библиотеки.

In [1]:
import pandas as pd
from IPython.display import display
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Прочитаем файл data.csv и сохраним его в переменной df.

In [2]:
df = pd.read_csv('/datasets/data.csv')

**1.3 Изучение общей информации** <a id="main_info"></a>

Получим общую информацию о данных таблицы df.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Количество значений в столбцах 'days_employed' и 'total_income' отличается от остальных, что говорит о том, что в данных о трудовом стаже и месячном доходе есть пропущенные значения.  

**1.4 Проверка корректность наименований столбцов** <a id="columns_info"></a>

Сразу проверим корректность наименований столбцов.

Столбцы названы корректно по следующм критериям: пробелы в начале отсутствуют, названия из 2х и более слов записаны через нижнее подчркивание, набраны в одном регистре, использован один язык (латиница), во всех столбцах в названиях в краткой форме отражена суть содержащейся в столбцах информации.
Получим перечень названий столбцов, чтобы убедиться в отсутствие пробелов в конце названий:

In [4]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

В конце названий столбцов так же не содержится пробелов.

**1.5 Изучение внешнего вида таблицы** <a id="main_view"></a>

Выведем на экран первые 20 строк таблицы.

In [5]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


По результатам визуального изучения данных можно отметить:

 1) Формат отображения данные в столбцах 'days_employed' и 'total_income' требует корректировок. Количество часов в трудовом стаже и копейки в ежемесячном доходе не сыграют большой роли. Уместнее использовать целочисленный тип данных.
 
 2) В информации о трудовом стаже, кроме пропусков, имеются отрицательные значения, что явно является ошибкой и требует исправления.
 
 3) Есть дубликаты в столбце 'education' из-за применения разных регистров. Так же нужно проверить на дубликаты данные в столбцах 'family_status' и 'gender'.
 
 4) Данные в столбце 'propuse' требуют леммитизации, т.к. похожие цели указаны по-разному. 

**1.6 Изучение статистических данных** <a id="stat_info"></a>

Выведем и изучим статистические сведения для количественных переменных нашей таблицы в целях выявления нестандартных значений и артефактов. 

In [6]:
df[['children', 'dob_years', 'days_employed']].agg(['mean', 'median', 'min', 'max'])

,children,dob_years,days_employed
mean,0.538908,43.29338,63046.497661
median,0.000000,42.00000,-1203.369529
min,-1.000000,0.00000,-18388.949901
max,20.000000,75.00000,401755.400475


In [7]:
print ('Среднее значение трудового стажа в годах:', int(df['days_employed'].mean()))
print ('Медиана трудового стажа в годах:', int(df['days_employed'].median()))
print ('Максимальное значение трудового стажа в годах:', int(df['days_employed'].max()))

Среднее значение трудового стажа в годах: 63046
Медиана трудового стажа в годах: -1203
Максимальное значение трудового стажа в годах: 401755


По столбцу 'children' следует проверить строки как с минимальным (-1), так и максимальным (20) значениями. Возможо, что имели место ошибки при занесении данных: 20 - слишком большое количество, (- 1) в принцbпе не возможное. 

В столбце 'dob_years' есть отсутствующие значения, т.к. возраст заёмщика не может быть равен нулю (минимальное значение - ноль). Можно так же проверить, есть ли значения меньше 18 лет.

По столбцу 'days_employed' наблюдается сильный разброс в данных (медиана и среднее сильно отличаются). Нереалистичные среднее и максимальное значения стажа: 63 и 401 тысячи соответственно. Возможно, часть данных о стаже заносились не в дня или на каком-то этапе данные были повреждены и искажены. Такие данные не пригодны для анализа. С учетом поставленных для анализа целей данные о стаже не являются обязательными и необходимыми. На этапе обработки пропусков удалим из таблицы столбец 'days_employed'.

**1.7 Дубликаты и уникальные значения** <a id="unique_info"></a>

In [8]:
print('Список значений, содержащиеся в столбцах', '\n')
print('education:', df['education'].unique(), '\n')
print('education_id:', df['education_id'].unique(), '\n')
print('family_status:', df['family_status'].unique(), '\n')
print('family_status_id:', df['family_status_id'].unique(), '\n')
print('gender:', df['gender'].unique(), '\n')
print('income_type:', df['income_type'].unique(), '\n')
print('debt:', df['debt'].unique())

Список значений, содержащиеся в столбцах 

education: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень'] 

education_id: [0 1 2 3 4] 

family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем'] 

family_status_id: [0 1 2 3 4] 

gender: ['F' 'M' 'XNA'] 

income_type: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете'] 

debt: [0 1]


Приведем символы к нижему регистру в столбце 'education', 'family_status' и проверим артефакт в списке 'gender'. 

In [9]:
df['education']=df['education'].str.lower() 
df['family_status']=df['family_status'].replace('Не женат / не замужем', 'не женат / не замужем')

print(df['education'].unique())
print(df['family_status'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


In [10]:
df.loc[df['gender']=='XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


По косвенным признакам нельзя определить какой пол должен быть указан. Для нашего анализа пол так же не имеет принципиального значения. Оставим как есть.

Проверим есть ли дубликаты по строкам и найдем их количество:

In [11]:
df.duplicated().sum()

71

**1.8 Вывод** <a id="main_conclusion"></a>

По результатам первичного исследования данных необходимо будет провести следующие этапы предобработки: 

1) Проверить артефакт с минимальным и максимальным количеством детей; 

2) Обработать пропуски: 
    * возраст меньше 18 лет и значения "ноль" 
    * удалить столбец 'days_employed'
    * NaN в столбце 'total_income'

3) Заменить вещественного типа на целочисленный в столбце 'total_income'

4) Проверить и удалить дубликаты

5) Лемматизировать данные в столбце 'propuse', т.к. похожие цели указаны по-разному

6) Категоризаровать данные исходя из заданных нам вопросов для анализа по количеству детей, доходу и целям кредита.
 



Причиной пропусков в столбцах 'days_employed' и 'total_income' может быть выдача кредита без предоставления справки о доходах или других документов, подтверждающих платежеспособность заемщика. Менее вероятной кажется причина, что сотрудник банка мог забыть заполнить именно эти 2 колонки, т.к. остальные данные заполнены и показатели месячного дохода и стажа важны для определения процентной ставки по кредиту. 

Появление дубликатов по регистрам в 'education' могло быть вызвано сбоем в программе, т.к. отличия есть только в регистре, т.е. данные скорее всего выбирались из закрытого перечня, а не вбивались руками, иначе список был бы шире и могли бы быть опечатки и более неявные различия (дополнительные пробелы, точки, и т.п.) или же из-за сбора информации из различных источников или программ, или из аналогичных программ, но с разными настройками (например, в разных филиалах банка могут быть различия в настройках), например, в одной программе информация хранится с регистром строчными буквами, а в другой: заглавными.

Возможно, в целях улучшения качества информации, стоит подумать над дополнительной проверкой унификации вносимых данных и провести такую проверку с коллегами, ответственными за такие вопросы, а так же внедрить проверку правильности заполнения трудового стажа - должен быть больше или равен нулю и не превышать, например, 40-50 лет.

## 2. Предобработка данных <a id="preprocessing"></a>

**2.1 Артефакты** <a id="artifact"></a>

**Минимальное и максимальное количество детей**

In [12]:
#сгруппируем данные и посмотрм как часто встречается то или иное количество детей 
print(df.groupby('children')['children'].count()) 
 
print('Общее количество строк:', df['children'].count())

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64
Общее количество строк: 21525


Количество строк с (- 1) - 47, c 20 - 76. Количество строк со значением 1 - 4814, 2 - 2055б 0 - 14149. 
Предположим, что (-1) на самом деле 1, а 20 на самом деле 0 и произведем замену. 
Даже если наше предположение ошибочно, соотношение изменяемых данных не столь велико, чтобы сильно исказить результаты. 

In [13]:
df['children']=df['children'].replace(-1, 1)
df['children']=df['children'].replace(20, 0)

#проверим сгруппированные данные и не изменилось ли общее количество:
print(df.groupby('children')['children'].count())
print('Общее количество строк:', df['children'].count())

children
0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64
Общее количество строк: 21525


**Вывод**

Проанализировали данные из таблицы с подозрительными значениями количества детей и исправили данные на более достоверные. 

**2.2 Обработка пропущенных значений** <a id="null"></a>

**2.2.1 Возраст меньше 18 лет и "ноль"**

Проверим нет ли в таблице заёмщиков младше 18 лет. 

In [14]:
df[(df['dob_years'] <= 18)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,-108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Посчитаем количество строк, где возраст указан ноль

In [15]:
df[(df['dob_years'] == 0)].groupby('dob_years')['dob_years'].count() 

dob_years
0    101
Name: dob_years, dtype: int64

Количество строк с возрастом "ноль" и "меньше 18" совпало. 
Сгруппируем отфильтрованные данные по типам занятости и полу.

In [16]:
df[(df['dob_years'] == 0)].groupby(['income_type'])['income_type'].count()

income_type
госслужащий     6
компаньон      20
пенсионер      20
сотрудник      55
Name: income_type, dtype: int64

Безработных среди "безвозрастных" нет. Посмотрим максимальное, среднее и медиану по возрасту. 

In [17]:
print(df['dob_years'].max())
print('{:.1f}'.format(df['dob_years'].mean()))
print(df['dob_years'].median())

75
43.3
42.0


Медиана меньше, чем среднее, значит есть значения, которые выделяются из общей массы. В таком случае лучше использовать медиану для заполнения пропусков. Заменим нулевые значения возраста по медиане исходя из группировки по типам занятости.

In [18]:
for income_type in df['income_type'].unique():
    median = df.loc[df['income_type'] == income_type, 'dob_years'].median() 
    print(income_type, median)  
    df.loc[(df['dob_years'] == 0) & (df['income_type'] == income_type), 'dob_years'] = median

#проверим остались ли строчки с нулевым значением возраста:
print(df[(df['dob_years'] == 0)])

сотрудник 39.0
пенсионер 60.0
компаньон 39.0
госслужащий 40.0
безработный 38.0
предприниматель 42.5
студент 22.0
в декрете 39.0
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


**2.2.2 NaN в столбце total_income**

Месячный доход - количественная переменная, пропуски будем заполнять характерными значениями - медианой. 
Заполним пропуски в 'total_income' исходя из данных по уровню образования и типу занятости. 
Для этого сгруппирум таблицу по соответствующим столбцам и найдем мериану месячного дохода.
С помощью метода .merge() добавим столбец с медианой по месячному доходу в нашу таблицу. 

In [19]:
print('Пропуски до:', df['total_income'].isna().sum())

Пропуски до: 2174


In [20]:
medians = (df.groupby(['education', 'income_type']).agg({'total_income': 'median'}).rename(columns = {'total_income': 'median_total_income'}))
df = df.merge(medians, on = ['education', 'income_type'])
print(df[['education', 'income_type', 'total_income', 'median_total_income']][df['total_income'].isna()].head())

   education income_type  total_income  median_total_income
12    высшее   сотрудник           NaN        165640.744634
13    высшее   сотрудник           NaN        165640.744634
16    высшее   сотрудник           NaN        165640.744634
28    высшее   сотрудник           NaN        165640.744634
34    высшее   сотрудник           NaN        165640.744634


In [21]:
df.loc[df['total_income'].isna(), 'total_income'] = df.loc[df['total_income'].isna(), 'median_total_income']
print('Пропуски после:', df['total_income'].isna().sum())

Пропуски после: 0


In [22]:
del df['median_total_income'] #удалим столбец с медианой,для дальнейших действий он нам больше не понадобится

**2.2.3 Удалить столбец days_employed**

По результатам изучений данных решили удалить столбец 'days_employed', т.к. не будем использовать его для анализа. 

In [23]:
df.dropna(axis='columns', inplace = True)

In [24]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,2,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
2,0,33.0,высшее,0,гражданский брак,1,M,сотрудник,0,90410.586745,строительство недвижимости
3,0,21.0,высшее,0,гражданский брак,1,M,сотрудник,0,128265.720871,сыграть свадьбу
4,0,28.0,высшее,0,женат / замужем,0,M,сотрудник,0,308848.983691,строительство собственной недвижимости


**Вывод**

В итоге обработали все пропущенные значения разных типов: неожиданные, такие как нули в данных о возрасте, ожидаемые, как NaN в ежемесячном доходе. Для этого использовали метод заполнения пропусков на основе расчета медианы по одному и нескольким значениям. 
Избавлись от некорректных данных со стажем, которые не будем использовать в анализе.

**2.3 Замена типа данных** <a id="type"></a>

В связи с тем, что текущая точность в столбце 'total_income' некорректна с точки зрения единиц измерения (логично было бы оставить 2 цифры после запятой - копейки, но для наших целей и они ни к чему), заменим вещественный тип float64 на целочисленный int. Так как мы уже удалили из столбца все NaN, то можно использовать метод .astype(). 

In [25]:
df['total_income']=df['total_income'].astype('int')
df.dtypes

children              int64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
dtype: object

**Вывод**

Теперь у всех данных подходящий тип. 

**2.4 Обработка дубликатов** <a id="duplicates"></a>

Выведем таблицу с дубликатами, отсортированную по показателю ежемесячного дохода и цели кредита, используя стандартный метод 'duplicated()', который подходит для "грубых" дубликатов, которые мы и ищем:

In [26]:
df[df.duplicated(keep=False)].sort_values(by=['total_income', 'purpose'])

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
11808,0,64.0,среднее,1,женат / замужем,0,F,пенсионер,0,114842,автомобиль
12578,0,64.0,среднее,1,женат / замужем,0,F,пенсионер,0,114842,автомобиль
12390,0,64.0,среднее,1,женат / замужем,0,F,пенсионер,0,114842,дополнительное образование
13623,0,64.0,среднее,1,женат / замужем,0,F,пенсионер,0,114842,дополнительное образование
11147,0,64.0,среднее,1,женат / замужем,0,F,пенсионер,0,114842,жилье
...,...,...,...,...,...,...,...,...,...,...,...
20539,2,36.0,высшее,0,женат / замужем,0,F,госслужащий,0,172511,получение образования
14493,0,38.0,высшее,0,гражданский брак,1,F,компаньон,0,201785,на проведение свадьбы
15346,0,38.0,высшее,0,гражданский брак,1,F,компаньон,0,201785,на проведение свадьбы
13924,0,54.0,высшее,0,женат / замужем,0,M,компаньон,0,201785,операции с коммерческой недвижимостью


Судя по таблице, строки действительно являются дубликатами. Их можно удалить стандартным методов drop_duplicates() с использованием методам reset_index() для упорядочивания нумерации индексов после удаления строк. 

In [27]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum() #проверим количество дубликатов после удаления

0

**Вывод**

Убрали из таблицы дубликаты. Причинами появления 'грубых' дубликатов может быть сбор информации из разных источников, программ. 
Они могли повлиять на результаты нашего анализа в связи с искусственным увеличением показателей в разрезе категорий, что привело бы к не верным выводам. 

**2.5 Лемматизация** <a id="lemma"></a>

Посчитаем частоту, с которой встречаются слова в столбце 'purpose' для выявления основных причин взятия кредита. Для этого соберем все уникальные значения из столбца 'purpose', объеденим их в строку и лемматизируем. В помощью контейнера Counter посчитаем количество лемматизированных слов и отфильтруем по возрастанию с помощью метода .most_common() 

In [28]:
Counter(m.lemmatize(' '.join(df['purpose'].unique()))).most_common()

[(' ', 96),
 ('покупка', 10),
 ('недвижимость', 10),
 ('образование', 9),
 ('автомобиль', 9),
 ('жилье', 7),
 ('с', 5),
 ('на', 4),
 ('свой', 4),
 ('операция', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('высокий', 3),
 ('получение', 3),
 ('сделка', 2),
 ('жилой', 2),
 ('коммерческий', 2),
 ('для', 2),
 ('заниматься', 2),
 ('дополнительный', 2),
 ('проведение', 1),
 ('сыграть', 1),
 ('собственный', 1),
 ('подержать', 1),
 ('профильный', 1),
 ('подержанный', 1),
 ('со', 1),
 ('приобретение', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('семья', 1),
 ('\n', 1)]

Можно выделить следующие 4 причины кредитования: жилье и недвижимость, свадьба, образование, автомобиль.

Для последующей категоризации добавим в таблицу новый столбец 'purpose_lemmas' с лемматизированным списком слов. 

In [29]:
df['purpose_lemmas']=df['purpose'].apply(m.lemmatize) 

In [30]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,2,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
2,0,33.0,высшее,0,гражданский брак,1,M,сотрудник,0,90410,строительство недвижимости,"[строительство, , недвижимость, \n]"
3,0,21.0,высшее,0,гражданский брак,1,M,сотрудник,0,128265,сыграть свадьбу,"[сыграть, , свадьба, \n]"
4,0,28.0,высшее,0,женат / замужем,0,M,сотрудник,0,308848,строительство собственной недвижимости,"[строительство, , собственный, , недвижимост..."


**Вывод**

Провели лемматизацию по целям кредита, что пригодится нам для дальнейшей категоризации и анализа.

**2.6 Категоризация данных** <a id="category"></a> 

**2.6.1 Категоризация по цели кредита**

Проведем категоризацию по целям кредита, которые мы леммитизировали в предыдущем пункте.

In [31]:
def purpose_group(purpose_lemmas):
    if 'жилье' in purpose_lemmas:
        return 'недвижимость'
    elif 'ремонт' in purpose_lemmas:
        return 'недвижимость'
    elif 'недвижимость' in purpose_lemmas:
        return 'недвижимость'
    elif 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    elif 'образование' in purpose_lemmas:
        return 'образование'
    elif 'свадьба' in purpose_lemmas:
        return 'свадьба'
    return 'нет подходящей категории'

df['purpose_group']= df['purpose_lemmas'].apply(purpose_group)

In [32]:
#функция для присвоения id целям кредита для создания словаря
def purpose_group_id(purpose_group_id):
    if 'недвижимость' in purpose_group_id:
        return '0'
    elif 'автомобиль' in purpose_group_id:
        return '1'
    elif 'образование' in purpose_group_id:
        return '2'
    elif 'свадьба' in purpose_group_id:
        return '3'
    return 'нет подходящей категории'

df['purpose_group_id']=df['purpose_group'].apply(purpose_group_id)

In [33]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_group,purpose_group_id
0,1,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,0
1,2,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,3
2,0,33.0,высшее,0,гражданский брак,1,M,сотрудник,0,90410,строительство недвижимости,"[строительство, , недвижимость, \n]",недвижимость,0
3,0,21.0,высшее,0,гражданский брак,1,M,сотрудник,0,128265,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,3
4,0,28.0,высшее,0,женат / замужем,0,M,сотрудник,0,308848,строительство собственной недвижимости,"[строительство, , собственный, , недвижимост...",недвижимость,0


Проверим все ли цели попали в подходящие категории и всем ли целям присвоены id:

In [34]:
df[df['purpose_group'] == 'нет подходящей категории']['purpose_group'].count()

0

In [35]:
df[df['purpose_group_id'] == 'нет подходящей категории']['purpose_group_id'].count()

0

Выделим из общей таблицы словарь. 

In [36]:
#словарь по целям кредита
purpose_group_dict = df[['purpose_group','purpose_group_id']]
purpose_group_dict = purpose_group_dict.drop_duplicates().reset_index(drop=True)
print(purpose_group_dict.sort_values(by='purpose_group_id', ascending = True))

  purpose_group purpose_group_id
0  недвижимость                0
3    автомобиль                1
2   образование                2
1       свадьба                3


Словари удобно использовать, если таблица содержит много длинных строк. Такие строки создают визуальный шум и затрудняют работу с данными, повышается возможность допустить опечатку при вводе названий. В нашем случае таких данных нет и пользователям при просмотре результатов анализа будет удобнее ориентироваться на словесное описание. 

**2.6.2 Категоризация по доходу**

В связи с тем, что довольно сложно выбрать базу для деления величины дохода на категории (например, данные о доходе среднего класса разнятся в зависимости от методологий расчета и принимаемых во внимание признаков такого класса), воспользуемся методов 'qcut'(), который разделит весь наш список доходов на категории таким образом, чтобы количество значений, попавших в каждый, было одинаковым. 

In [37]:
df['qcut_total_income'] = pd.qcut(df['total_income'], q=5, labels=['a_little_bit', 'not_so_good', 'medium', 'good', 
                                                                   'excellent'])

**2.6.3 Категоризация по количеству детей**

Так же нам нужно установить есть ли связь между наличием детей и возвратом кредита. 
Сгруппируем данные по количеству детей следующим образом:

1) без детей

2) 1-2 ребёнка

3) многодетные. 

Для этого напишем функцию и применим её к таблице.

In [38]:
def child_group (children):
    if children == 0:
        return 'без детей'
    elif children >= 3:
        return 'многодетные'
    elif 0 < children < 3:
        return '1-2 ребёнка'
    return 'проверить корректность введенных данных'

df['child_group']=df['children'].apply(child_group)

In [39]:
#проверим работу функции
child_group(-5)

'проверить корректность введенных данных'

**Вывод**

Категоризовали данные по целям кредита, группам ежемесячного дохода и количеству детей.

### 3. Ответы на вопросы <a id="answers"></a>

**3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?** <a id="answers"></a>

In [40]:
child_df = df.pivot_table(index=['child_group'], columns = 'debt', values ='total_income', aggfunc='count')
child_df.columns = ['no_debt', 'debt']
child_df['return percentage'] = child_df['debt'] / (child_df['debt'] + child_df['no_debt'])
child_df.style.format({'return percentage' : '{:.2%}'})

,no_debt,debt,return percentage
child_group,,,
1-2 ребёнка,6268,639,9.25%
без детей,13096,1071,7.56%
многодетные,349,31,8.16%


**Вывод**

Самыми ответственными оказались заёмщики без детей. А заёмщики с 1 или 2 детьми чаще склоны не возвращать кредит в срок. Возможно сказывается желание излишне побаловать чадо, ведь оно одно единственное или всего лишь второе. Эти же можно было бы объяснить средние показатели многодетных, в большой семье уже не до баловства! В действительности же нельзя казать, что прослеживается однозначная корреляция между количеством детей и соблюдением срока возврата кредита. 

**3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?** <a id="answers_family"></a>

In [41]:
family_status_df = df.pivot_table(index=['family_status'], columns = 'debt', values ='total_income', aggfunc='count')
family_status_df.columns = ['no_debt', 'debt']
family_status_df['return percentage'] = family_status_df['debt'] / (family_status_df['debt'] + family_status_df['no_debt'])
family_status_df.style.format({'return percentage' : '{:.2%}'})

,no_debt,debt,return percentage
family_status,,,
в разводе,1110,85,7.11%
вдовец / вдова,896,63,6.57%
гражданский брак,3763,388,9.35%
женат / замужем,11408,931,7.55%
не женат / не замужем,2536,274,9.75%


**Вывод**
Заёмщики, не вступавшие в официальный брак (не женат/не замужем и гражданский брак), более склоны к просрочкам по кредитам. Интересно, что те, кто "прошли" семейную жизнь (в разводе, вдовы/вдовцы) более щепетильны в вопросах погашения задолженности. 

Возможно стоит попробовать оценить влияние и семейного положения и наличия детей.

In [42]:
status_child_df = df.pivot_table(index=['family_status', 'child_group'], columns = 'debt', values ='total_income', 
                                        aggfunc='count') 
status_child_df.columns = ['no_debt', 'debt']
status_child_df['return percentage'] = status_child_df['debt'] / (status_child_df['debt'] + status_child_df['no_debt'])
status_child_df.style.format({'return percentage' : '{:.2%}'})

Самые беспечными оказались многодетные и не состоящие в браке. Таким лучше кредит не давать! Радует, что таких не много. 
Больше всего гарантий дождаться возврата денег от состоящих в официальном браке и не имеющим при этом детей, так же можно доверять и вдовам/вдовцам без детей. 
Подтверждается факт того, что самый низкий процент невозвратов у заёмщиков без детей.    

**3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?** <a id="answers_income"></a>

In [43]:
income_group_df = df.pivot_table(index=['qcut_total_income'], columns = 'debt', values ='total_income', aggfunc='count')
income_group_df.columns = ['no_debt', 'debt']
income_group_df['return percentage'] = income_group_df['debt'] / (income_group_df['debt'] + income_group_df['no_debt'])
income_group_df.style.format({'return percentage' : '{:.2%}'})

,no_debt,debt,return percentage
qcut_total_income,,,
a_little_bit,3947,344,8.02%
not_so_good,3932,359,8.37%
medium,3904,386,9.00%
good,3939,352,8.20%
excellent,3991,300,6.99%


**Вывод**

Как ни странно, больше всего нарушают те, чьи доходы не слишком высокие и не слишком низкие, а самые сознательные те, кто получает больше остальных. Видимо сказывается умение управлять деньгами...  

**3.4 Как разные цели кредита влияют на его возврат в срок?** <a id="answers_purpose"></a>

In [44]:
purpose_group_df = df.pivot_table(index=['purpose_group'], columns = 'debt', values ='total_income', aggfunc='count')
purpose_group_df.columns = ['no_debt', 'debt']
purpose_group_df['return percentage'] = purpose_group_df['debt'] / (purpose_group_df['debt'] + purpose_group_df['no_debt'])
purpose_group_df.style.format({'return percentage' : '{:.2%}'})

,no_debt,debt,return percentage
purpose_group,,,
автомобиль,3903,403,9.36%
недвижимость,10029,782,7.23%
образование,3643,370,9.22%
свадьба,2138,186,8.00%


**Вывод**

Лучше всего возвращают банку деньги за недвижимость. Видимо, потому что если не возвращать, то банк может эту недвижимость и отобрать, но и берут тоже в основном для этой же цели. 
Хуже всего отдают деньги, потраченные на образование (его не отберешь!) и автомобиль (быстро теряет рыночную стоимость).

###  4. Общий вывод <a id="conclusion"></a>

Самая популярная цель кредитования: недвижимость, что объяснимо высокими ценами на неё. Но и погашают такие кредиты охотнее. 
Идеальный заёмщик: состоящий (или состоявший ранее) в браке , но без детей, с высоким уровнем дохода, берущий кредит на недвижимость.
Больше всего риск просрочки от заемщиков: не состоящих в официальном браке, но с 1-2 детьми, со средним уровнем дохода с целью приобретения  автомобиля. 